# Downloads

In [ ]:
pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 5.4 MB/s 


# Imports

In [ ]:
import zipfile
import pandas as pd
import numpy as np
import matplotlib as plt
import random as rd
import time
import os
import openpyxl
import xlsxwriter
import math

# Main

In [ ]:

writer1= pd.ExcelWriter('Hibrido.xlsx',engine='xlsxwriter')
k=3
s= zipfile.ZipFile('pmedcap.zip','r')
Zipfile = ('pmedcap.zip','r')
with s as zipObj:
  listOfiles = zipObj.namelist()
  print(listOfiles)
  for elem in listOfiles:
    txtdata = s.extract(elem)
    listN = []
    with open(txtdata) as f:
      lines = f.readline()
      print(lines)
      list1 = list(lines.split("\t"))
      print(list1)
      lista = list(list1[0:2])
      print(lista)
      results = [int(i) for i in lista]
      while lines:
        lines = f.readline()
        if lines == "":
          break
        else:
          list1 = list(lines.split("\t"))
          lista = [int(i) for i in list1]
          lista = np.array(lista)
          listN.append(lista)

    listNodos = np.array(listN.copy())
    numNodos = results[0]
    numPmedians = results[1]
    hib = hibrido(listNodos, numPmedians, numNodos)
    print(elem)
    hib.to_excel(writer1,sheet_name=elem)
writer1.save()


['pmedcap1.txt', 'pmedcap2.txt']
50	5		

['50', '5', '', '\n']
['50', '5']
True
pmedcap1.txt
50	5		

['50', '5', '', '\n']
['50', '5']
True
pmedcap2.txt


# Main printing

In [ ]:
s= zipfile.ZipFile('pmedcap.zip','r')
Zipfile = ('pmedcap.zip','r')
with s as zipObj:
  listOfiles = zipObj.namelist()
  #print(listOfiles)
  for elem in listOfiles:
    txtdata = s.extract(elem)
    listN = []
    with open(txtdata) as f:
      lines = f.readline()
      #print(lines)
      list1 = list(lines.split("\t"))
      #print(list1)
      lista = list(list1[0:2])
      #print(lista)
      results = [int(i) for i in lista]
      while lines:
        lines = f.readline()
        if lines == "":
          break
        else:
          list1 = list(lines.split("\t"))
          lista = [int(i) for i in list1]
          lista = np.array(lista)
          listN.append(lista)

    listNodos = np.array(listN.copy())
    numNodos = results[0]
    numPmedians = results[1]
    print(elem)
    hib = hibrido(listNodos, numPmedians, numNodos)
    print(hib)
    

pmedcap1.txt
True
      0           1     2     3     4     5     6     7     8     9      10  \
0   13.0   10.000000  13.0   2.0   6.0   8.0   9.0  20.0  35.0  40.0   43.0   
1   20.0   12.000000  20.0   4.0   5.0  22.0  24.0  27.0  28.0  29.0   31.0   
2   33.0   12.000000  33.0   1.0   3.0  11.0  14.0  21.0  39.0  41.0   44.0   
3   34.0    9.000000  34.0  15.0  16.0  18.0  26.0  34.0  36.0  48.0  135.0   
4   39.0   12.000000  39.0   7.0  10.0  13.0  17.0  23.0  25.0  30.0   42.0   
5  766.0  205.641069   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN   

      11     12     13     14  15  
0  109.0  109.0    NaN    NaN NaN  
1   37.0   47.0  141.0  141.0 NaN  
2   46.0   50.0  203.0  203.0 NaN  
3  135.0    NaN    NaN    NaN NaN  
4   45.0   49.0  178.0  178.0 NaN  
5    NaN    NaN    NaN    NaN NaN  
pmedcap2.txt
True
      0           1     2    3     4     5     6     7     8     9     10  \
0   17.0   11.000000  17.0  6.0  10.0  18.0  24.0  27.0  30.0  37.0  40.0   
1   

KeyboardInterrupt: ignored

# Llamador algoritmo

In [ ]:
def hibrido(listNodos, numPmedians, numNodos):
  startTime = time.time()
  listDistances = matrizDistancias(listNodos, numNodos, numPmedians)
  padres, anexos, listasNodos, fos = seleccion(numPmedians, numNodos, 30, 20, listDistances)
  while time.time() - startTime < 60:
    padres, anexos, fos, listasNodos = crossover(padres, anexos, listasNodos, fos, numPmedians, numNodos, listNodos,  listDistances) 
  
  funcObj = np.amin(fos)
  index = np.where(fos == funcObj)[0][0]
  sol = padres[index]
  anex = anexos[index]
  lnod = listasNodos[index]
  
  fax = faxCheck(listNodos, numPmedians, numNodos, sol, anex, funcObj, listDistances)
  print(fax)

  linFo = calculoFo2(sol, anex, listDistances)
  sol = np.where(sol == 1)[0]
  for i in range(len(anex)):
    for j in range(len(anex[i])):
      anex[i][j] += 1
    linFo[i] -= 1
    sol[i] += 1
  
  ultimaFila = [np.sum(linFo) + numPmedians, time.time()- startTime]  
  dfout = formato(sol, anex, ultimaFila, numPmedians, linFo)
  return dfout





#Selección

In [ ]:
def seleccion(numPmedians, numNodos, numSolIn, percentToTake, listDistances):
  solIn = []
  tamSave = int(numSolIn/100 * percentToTake)

  for i in range(numSolIn):
    indexes = rd.sample(range(numNodos), numPmedians)
    posibles = np.zeros(numNodos)
    for i in indexes:
     posibles[i] = 1
    solIn.append(posibles)
  
  anexos = []
  listasNodos = []
  fos = []
  guardados = []
  guardadosAnexos = []
  guardadosListasNodos = []
  guardadosFos = []
  for i in solIn:
    anexo, fo, lnodo = calculoFo(i, listDistances, listNodos)
    anexos.append(anexo)
    fos.append(fo)
    listasNodos.append(lnodo)
  sort = sorted(range(len(fos)), key = lambda k: fos[k])
  for i in range(int(tamSave)):
    indice = sort[i]
    guardados.append(solIn[indice])
    guardadosAnexos.append(anexos[indice])
    guardadosListasNodos.append(listasNodos[indice])
    guardadosFos.append(fos[indice])
  return guardados, guardadosAnexos, guardadosListasNodos, guardadosFos

# CrossOver

In [ ]:
def crossover(padres, anexos, listasNodos, fos, numPmedians, numNodos, listNodos,  listDistances):
  padresIndices = []
  children = []
  anexos1 = []
  listasNodos1 = []
  fos1 = []
  for i in padres:
    padresInd = np.where(i != 0)[0]
    padresIndices.append(padresInd)

  for i in range(len(padresIndices)-1):
    for j in range(i, len(padresIndices)):
      repetidos = []
      dad = padresIndices[i]
      mom = padresIndices[j]
      for elem1 in dad:
        if elem1 in mom:
          repetidos.append(elem1)
      if len(repetidos) == 0:
        continue
      else:
        child = np.zeros(numNodos)
        child[repetidos] += 1
      comp = numPmedians - len(repetidos)
      cont = 0
      while cont < comp:
        rand = rd.randint(0, numNodos-1)
        if rand in repetidos:
          continue
        else:
          repetidos.append(rand)
          child[rand] += 1
          cont += 1
      anexo, fo, lnodo = calculoFo(child, listDistances, listNodos)
      startTime1 = time.time()
      while time.time() - startTime1 < 20:
        child, anexo, fo, lnodo = vecindarioEleccionPmeds(listNodos, numPmedians, numNodos, child, anexo, fo, listDistances, lnodo)
      children.append(child)
      anexos1.append(anexo)
      fos1.append(fo)
      listasNodos1.append(lnodo)
  
  return children, anexos1, fos1, listasNodos1

# Vecindario cambio pmed

In [ ]:
def vecindarioEleccionPmeds(listNodos, numPmedians, numNodos, posibles, anexoInicial, foInicial, listD, listNodosI):
  start_time1 = time.time()
  for i in range(len(posibles)):
    if posibles[i] == 1:
      for j in range(len(posibles)):
        if posibles[j] == 0:
          xp = posibles.copy()
          xp[i] = 0
          xp[j] = 1
          anexo, fo, listN = calculoFo(xp, listD, listNodos)
          #mejoria
          if fo < foInicial :
            return xp, anexo, fo, listN
          #Parada de emergencia
          if time.time()-start_time1 >= 30:
            return posibles, anexoInicial, foInicial, listNodosI
  return posibles, anexoInicial, foInicial, listNodosI

# Auxiliares

In [ ]:
def faxCheck(listNodos, numPmedians, numNodos, posibles, anexoInicial, foInicial, listD):
  aux = listNodos.copy()
  caps = []
  dems = 0
  for i in range(numPmedians):
    cent = anexoInicial[i][0]
    dem = aux[cent, 3] 
    for j in range(1, len(anexoInicial[i])):
      dem += aux[anexoInicial[i][j], 3]
    dems += dem
    leftCap = aux[cent, 2] - dem
    #print(leftCap)
    caps.append(leftCap)
  boole = True
  for i in caps:
    if i<0:
      boole = False
  return boole

In [ ]:
def calculoFo(vecino, listDistances, listNodos):
  posibles = np.where(vecino != 0 )[0]
  aux = listNodos.copy()
  ayuda = list(map(lambda el:[el], posibles.copy()))
  ayuda1 = posibles.copy()

  for j in range(numNodos):
    array = listDistances[:,j]
    sort = sorted(range(len(array)), key = lambda k: array[k])
    if j not in ayuda1:
      for v in sort:
            if ((aux[v, 2] >= aux[v, 3] + aux[j, 3]) and (v in ayuda1)):
              addTo = np.where(ayuda1 == v)[0][0]
              aux[v, 3] += aux[j, 3]
              ayuda[addTo].append(j)
              break
  
  
  fo = np.zeros(len(posibles))
  ayuda1 = np.array(posibles.copy())
  for i in posibles:
    ayu = np.where(ayuda1 == i)
    added = 0
    for j in ayuda[ayu[0][0]]:
      added += int(listDistances[i, j])
    fo[ayu[0][0]] += added
  sumaTotal = sum(fo)
  return ayuda, sumaTotal, aux

In [ ]:
def matrizDistancias(listNodos, numNodos, numPmedians):
  aux = listNodos.copy()
  listDistances = np.zeros([numNodos, numNodos])
  for i in range(numNodos):
    listDistance = np.zeros(numNodos)
    for n in range( numNodos):
      distance = distancia(aux[i, 0], aux[n, 0], aux[i, 1], aux[n, 1]) 
      listDistance[n] = distance
    listDistances[i] += listDistance
  return listDistances

In [ ]:
def distancia(a1, a2, b1, b2):
  r = np.sqrt((a1-a2)**2+(b1-b2)**2)
  return r

In [ ]:
def calculoFo2(vecino, anexados, listDistances):
  posibles = np.where(vecino != 0 )[0]
  fo = np.zeros(len(posibles))
  ayuda1 = np.array(posibles.copy())
  for i in posibles:
    ayu = np.where(ayuda1 == i)
    added = 0
    for j in anexados[ayu[0][0]]:
      added += int(listDistances[i, j])
    fo[ayu[0][0]] += added
  return fo

In [ ]:
def formato(bases, anexos, ultimaFila, numPmedians, fos): 
  cuantosHay = []
  for i in range(len(anexos)):
    anexos[i].append(fos[i])
  for i in anexos:
    cuantosHay.append(len(i) - 1)
  df= np.empty((numPmedians+1,max(cuantosHay)+4))
  df[:]= np.nan
  df= pd.DataFrame(df) #creo el data frame
  for i in range(numPmedians):
    df.loc[i,0]= bases[i] + 1
    df.loc[i,1]= cuantosHay[i]
    for j in range(len(anexos[i])):
      df.loc[i,j+2]= int(anexos[i][j]) + 1
  df.loc[numPmedians,0]= int(ultimaFila[0])
  df.loc[numPmedians,1]= ultimaFila[1] 
  return df